In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_694915/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_45_0,0.995686,0.801012,0.776185,0.991193,0.017956,1.330632,0.800574,0.029338,0.311930,0.133999,1.276073,0.136109,138.039705,279.374878,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_45_1,0.995608,0.801043,0.776995,0.990377,0.018283,1.330426,0.797676,0.032057,0.309470,0.135216,1.281112,0.137346,138.003525,279.338697,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_45_2,0.995538,0.801099,0.777924,0.989630,0.018572,1.330049,0.794354,0.034543,0.306268,0.136281,1.285556,0.138427,137.972156,279.307328,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_45_3,0.995452,0.801085,0.778656,0.988795,0.018932,1.330148,0.791736,0.037326,0.303865,0.137595,1.291092,0.139762,137.933755,279.268927,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_45_4,0.995393,0.801070,0.779164,0.988123,0.019177,1.330245,0.789916,0.039562,0.301307,0.138481,1.294850,0.140662,137.908096,279.243269,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,model_1_5_4,0.892474,0.671480,0.748408,0.938737,0.447578,2.196817,0.899929,0.299638,0.878413,0.669013,7.881635,0.679549,131.607809,272.942981,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
446,model_1_5_3,0.890199,0.669240,0.745981,0.937081,0.457051,2.211795,0.908611,0.307743,0.886785,0.676055,8.027282,0.686702,131.565922,272.901094,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
447,model_1_5_2,0.887899,0.666991,0.743526,0.935403,0.466623,2.226833,0.917392,0.315949,0.895039,0.683098,8.174463,0.693856,131.524466,272.859638,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
448,model_1_5_1,0.885541,0.664721,0.740937,0.933636,0.476437,2.242012,0.926654,0.324589,0.903473,0.690244,8.325344,0.701115,131.482841,272.818014,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
